In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import nltk.sentiment
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np

import os
import unicodedata
import re
import json

import j_acquire
import j_prep

In [2]:
df = j_acquire.scrape_github_data()

In [3]:
df = pd.DataFrame(df)
df

,repo,language,readme_contents
0,openai/jukebox,Python,"**Status:** Archive (code is provided as-is, n..."


In [4]:
df['readme_contents'] = df['readme_contents'].str.lower()

In [5]:
df['link_counts'] = df.readme_contents.str.count(r'https*').sum()

In [6]:
df.head()

,repo,language,readme_contents,link_counts
0,openai/jukebox,Python,"**status:** archive (code is provided as-is, n...",7


In [7]:
# re.findall(r'\[.*?\]', df.readme_contents)

In [8]:
# for line in df.readme_contents:
#     re.findall(r'\[.*?\]')


In [9]:
# contents = df.readme_contents.to_string()

# # regex = '\[(.+)\]\(([^ ]+)( "(.+)")?\)' 

# len(contents)

In [10]:
def basic_clean(df, col):
    '''
    This function takes in a df and a string for a column and
    returns the df with a new column named 'basic_clean' with the
    passed column text normalized.
    '''
    df['basic_clean'] = df[col].str.lower()\
                    .replace(r'[^\w\s]', '', regex=True)\
                    .str.normalize('NFKC')\
                    .str.encode('ascii', 'ignore')\
                    .str.decode('utf-8', 'ignore')
    return df

In [11]:
# df = basic_clean(df, 'readme_contents')

In [12]:
df = j_prep.prep_article_data(df)

In [13]:
df.head()

,repo,language,readme_contents,link_counts,basic_clean,clean_tokes,lemmatized,clean_lemmatized
0,openai/jukebox,Python,"**status:** archive (code is provided as-is, n...",7,status archive code is provided asis no update...,"[status, archive, code, is, provided, asis, no...",status archive code is provided asis no update...,status archive code provided asis update expec...
